In [ ]:
pip install torch torchvision


     -------------------------------------- 192.3/192.3 MB 1.7 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 3.7 MB/s eta 0:00:00
     -------------------------------------- 166.4/166.4 kB 5.0 MB/s eta 0:00:00
     ---------------------------------------- 5.7/5.7 MB 1.2 MB/s eta 0:00:00
     -------------------------------------- 536.2/536.2 kB 2.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
import torch
from torch import nn
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import time
import copy
torch.set_num_threads(4)
torch.set_num_interop_threads(4)

## Load Data

This is an example of building a torch data loader for a dataset!

In [3]:
train_csv=pd.read_csv('Data/small_file.csv')
test_csv=pd.read_csv('Data/test.csv')
valid_csv=pd.read_csv('Data/val.csv')

In [4]:
class FashionDataset(Dataset):
    """User defined class to build a datset using Pytorch class Dataset."""
    
    def __init__(self, data, transform = None):
        """Method to initilaize variables.""" 
        self.fashion_MNIST = list(data.values)
        self.transform = transform
        
        label = []
        image = []
        
        for i in self.fashion_MNIST:
             # first column is of labels.
            label.append(i[0])
            image.append(i[1:])
        self.labels = np.asarray(label)
        # Dimension of Images = 28 * 28 * 1. where height = width = 28 and color_channels = 1.
        self.images = np.asarray(image).reshape(-1, 28, 28, 1).astype('float32')
        self.images = self.images/256

    def __getitem__(self, index):
        label = self.labels[index]
        image = self.images[index]
        
        if self.transform is not None:
            image = self.transform(image)

        return image, label

    def __len__(self):
        return len(self.images)

In [5]:
batch_size=256

train_set = FashionDataset(train_csv, transform=transforms.Compose([transforms.ToTensor()]))
val_set = FashionDataset(valid_csv, transform=transforms.Compose([transforms.ToTensor()]))
test_set = FashionDataset(test_csv, transform=transforms.Compose([transforms.ToTensor()]))

train_loader = DataLoader(train_set, batch_size=batch_size)
val_loader = DataLoader(val_set, batch_size=batch_size)
test_loader = DataLoader(test_set, batch_size=batch_size)

## Build a MLP

The number of input is decided by the images which is 28*28 = 784, the number of output is 10 which is 0 to 9. The size of hidden layer is a tunable hyperparameter. 

In [6]:
import torch.nn.functional as F
class MLP(nn.Module):
    def __init__(self,num_inputs=784,num_outputs=10,num_hiddens=256):
        super(MLP, self).__init__()
        
        
        self.hidden = nn.Linear(num_inputs, num_hiddens)
        
        self.output = nn.Linear(num_hiddens, num_outputs)
        
        
        

    def forward(self, X):
       
        X = X.view(X.size(0), -1)
        
        X = F.relu(self.hidden(X))
        
        X = self.output(X)

        return X
       
        

In [7]:
# function for evaluating the model performance
def eval_model(model,data_loader):
    model.eval()
    y_true_list=[]
    y_pred_list=[]
    model.eval()
    for x,y in data_loader:
        outputs=model(x)
        _, y_pred = torch.max(outputs, 1)
        y_pred_list.extend(y_pred.clone().detach().tolist())
        y_true_list.extend(y.clone().detach().tolist())
    acc=classification_report(y_true_list, y_pred_list,output_dict=True)['accuracy']
    return acc

## Train the MLP 



In [9]:
random_seed = 3407
torch.manual_seed(random_seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# hyperparameter values to consider.
hiddens=[256,512,1024]
lrs=[5e-2, 1e-1, 5e-1]
num_inputs=784
num_outputs=10
#loss function we gonna use 
loss_function = nn.CrossEntropyLoss()

In [10]:
import torch.optim as optim
import time

start_time=time.time()
current_best=0.0
best_model=None
criterion = nn.CrossEntropyLoss()
for h in hiddens:
    for lr in lrs:
      
        
        model = MLP(num_inputs=num_inputs, num_outputs=num_outputs, num_hiddens=h)
        optimizer = torch.optim.SGD(model.parameters(), lr=lr)
        
        
        
        for i in range(30):
           
            model.train()
           
            for x,y in train_loader:
                
                optimizer.zero_grad()
                
                outputs = model(x)
                
                loss = criterion(outputs, y)
                
                loss.backward()
                
                optimizer.step()
                
            if i%5 == 0:
                accuracy = eval_model(model, val_loader)
                print(f"Epoch {i}, Hidden units: {h}, Learning rate: {lr}, Accuracy: {accuracy:.4f}")
                # Check if current accuracy is greater than the previous best
                if accuracy > current_best:
                    current_best = accuracy
                    best_model = model
                    
               

Epoch 0, Hidden units: 256, Learning rate: 0.05, Accuracy: 0.6701
Epoch 5, Hidden units: 256, Learning rate: 0.05, Accuracy: 0.7885
Epoch 10, Hidden units: 256, Learning rate: 0.05, Accuracy: 0.7976
Epoch 15, Hidden units: 256, Learning rate: 0.05, Accuracy: 0.8035
Epoch 20, Hidden units: 256, Learning rate: 0.05, Accuracy: 0.8102
Epoch 25, Hidden units: 256, Learning rate: 0.05, Accuracy: 0.8192
Epoch 0, Hidden units: 256, Learning rate: 0.1, Accuracy: 0.7050
Epoch 5, Hidden units: 256, Learning rate: 0.1, Accuracy: 0.7818
Epoch 10, Hidden units: 256, Learning rate: 0.1, Accuracy: 0.8007
Epoch 15, Hidden units: 256, Learning rate: 0.1, Accuracy: 0.8132
Epoch 20, Hidden units: 256, Learning rate: 0.1, Accuracy: 0.8238
Epoch 25, Hidden units: 256, Learning rate: 0.1, Accuracy: 0.8312
Epoch 0, Hidden units: 256, Learning rate: 0.5, Accuracy: 0.5874
Epoch 5, Hidden units: 256, Learning rate: 0.5, Accuracy: 0.8117
Epoch 10, Hidden units: 256, Learning rate: 0.5, Accuracy: 0.8292
Epoch 15, 